In [97]:
import pandas as pd
import networkx as nx
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>{display: table-cell !important; text-align: center !important; .output_png vertical-align: middle !important;}</style>"))
CSS = """
.output {
    align-items: center;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [98]:
df_links = pd.read_excel("scigrid-conference-eu-data-only/links_eu_power_160718.csvdata.xlsx")
df_nodes = pd.read_excel("scigrid-conference-eu-data-only/vertices_eu_power_160718.csvdata.xlsx")

## Power stations - color depending on type, size depending on voltage

In [99]:
df_nodes.drop(['frequency', 'ref', 'operator'], axis=1, inplace=True)

In [119]:
G=nx.from_pandas_edgelist(df_links, 'v_id_1', 'v_id_2', 'voltage')
degrees = dict(G.degree)
types = df_nodes['typ'].values
station_voltage = (df_nodes['voltage'].values).tolist()
for i in range(len(station_voltage)):
    if isinstance(station_voltage[i], str):
        voltage_split = station_voltage[i].split(';')
        for j in range(len(voltage_split)):
            voltage_split[j] = int(voltage_split[j])
        station_voltage[i] = max(voltage_split)
        # station_voltage[i] = int(station_voltage[i])
    elif isinstance(station_voltage[i], int):
        pass
    else:
        station_voltage[i] = 11000
        
station_voltage = [float(i)/max(station_voltage) for i in station_voltage]

0.014666666666666666


In [121]:
fig = px.scatter_geo(df_nodes,
                lat=df_nodes.lat,
                lon=df_nodes.lon,
                color=types,
                color_continuous_scale=px.colors.sequential.Plasma,
                size=station_voltage,
                size_max = 10,
                hover_name="v_id")
fig.update_geos(
    center=dict(lon=df_nodes.lon.median(), lat=df_nodes.lat.median()),
    lataxis_range=[-15,15], lonaxis_range=[50, 100]
)
# code to add cities - may be cool to use later, lmk
# fig.add_trace(go.Scattergeo(
#                     lon = [9.189510],             
#                     lat = [45.4642715],
#                     text = 'Milano',
#                     marker_color='red',
#                     marker_size=8 ))

fig.update_layout(width = 800, height=600,
        title_text = 'PowerGrid of Europe',title_x=0.5,
        showlegend = True,
        geo = dict(
            scope = 'europe',
            resolution = 50,
            landcolor = '#abbad6',
        )
    )
fig.show()

# there are many stations with no voltage given - i passed a zero, but it doesn't work well with size scaling 
# i was thinking about assigning them the minimal value, wyt?

## Links - width depending on voltage